In [136]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

In [137]:
from pathlib import Path
p = Path().cwd()
print(f"original File path: {p}")
stringpath= str(p)[0:str(p).rfind('\\')] + '\\data'
print(f"Updated File Path to pick from Data Library: { stringpath}")
ngsimfile=stringpath + '/' + 'Next_Generation_Simulation__NGSIM__Vehicle_Trajectories_and_Supporting_Data.csv'
ngsim = pd.read_csv(ngsimfile,low_memory=False)
print(f"{ngsim.duplicated().sum()} duplicate values have been removed")
ngsim.drop_duplicates(inplace=True)

original File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\scripts
Updated File Path to pick from Data Library: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\data
704000 duplicate values have been removed


In [138]:
ngsim['v_length']= ngsim['v_length']/0.3048
ngsim['Space_Headway']= ngsim['Space_Headway']/0.3048
ngsim['v_Vel']= ngsim['v_Vel']/0.3048
ngsim['v_Acc']= ngsim['v_Acc']/0.3048
ngsim['Relative_Time']= ngsim['Global_Time'] - ngsim['Global_Time'].min() + 1
ngsim= ngsim.drop(columns=['Movement', 'Direction','Section_ID','Int_ID','D_Zone','O_Zone','Lane_ID','Following','v_Width','Total_Frames','Frame_ID'])

In [139]:
#filtered_ngsim= ngsim[((ngsim['Location'] == 'us-101') | (ngsim['Location'] == 'i-80')) & (ngsim['Preceding']>0) & (ngsim['Following']>0) ]
ngsim['Preceding_Vehicle_Class'] = np.NaN
ngsim["v_Class_Name"] = ngsim["v_Class"].map({1:"Motorcycle", 2: "Car", 3: "Heavy Vehicle"})

filtered_ngsim_U= ngsim[((ngsim['Location'] == 'us-101')) ]
filtered_ngsim_I= ngsim[((ngsim['Location'] == 'i-80')) ]

In [140]:
filtered_ngsim_U[((filtered_ngsim_U['Space_Headway'] == 0 ) & (filtered_ngsim_U['Preceding'] > 0 )) ]

,Vehicle_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,v_Acc,Preceding,Space_Headway,Time_Headway,Location,Relative_Time,Preceding_Vehicle_Class,v_Class_Name
76361,99,1118847948500,12.942,1420.624,6452132.084,1872437.889,27.887139,1,187.007874,-0.688976,92,0.0,0.0,us-101,1117684929401,NaN,Motorcycle
107058,806,1118848229800,6.699,796.216,6451666.005,1872853.511,47.572178,2,30.085302,0.000000,814,0.0,0.0,us-101,1117685210701,NaN,Car
452347,366,1118848971900,12.613,835.132,6451695.487,1872819.732,22.965879,1,93.602362,19.553806,370,0.0,0.0,us-101,1117685952801,NaN,Motorcycle
3595333,2871,1118847820000,7.631,547.070,6451508.896,1872991.251,42.650919,2,31.692913,0.000000,2876,0.0,0.0,us-101,1117684800901,NaN,Car
5900431,2964,1118847852000,4.868,309.729,6451336.355,1873154.258,54.133858,2,28.182415,-36.745407,2956,0.0,0.0,us-101,1117684832901,NaN,Car


In [141]:
filtered_ngsim_I[((filtered_ngsim_I['Space_Headway'] == 0 ) & (filtered_ngsim_I['Preceding'] > 0 )) ]

,Vehicle_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,v_Acc,Preceding,Space_Headway,Time_Headway,Location,Relative_Time,Preceding_Vehicle_Class,v_Class_Name
5910808,2506,1113437493200,53.343,380.297,6042837.478,2133451.781,46.91601,2,37.828084,-0.853018,2512,0.0,0.00,i-80,1112274474101,NaN,Car
6759942,2491,1113437558500,17.161,598.884,6042774.601,2133664.163,50.19685,2,37.171916,0.393701,2486,0.0,0.00,i-80,1112274539401,NaN,Car
9153569,1734,1113438171800,17.373,77.065,6042839.014,2133146.309,50.19685,2,82.020997,0.000000,1748,0.0,0.00,i-80,1112275152701,NaN,Car
9365776,572,1113437894100,17.928,1485.487,6042644.407,2134541.664,46.91601,2,0.000000,0.000000,566,0.0,9999.99,i-80,1112274875001,NaN,Car
9803270,1394,1113437169900,76.566,444.910,6042852.523,2133518.460,48.55643,2,20.078740,0.000000,1407,0.0,0.00,i-80,1112274150801,NaN,Car
10074152,1734,1113438174600,17.393,141.704,6042831.057,2133210.438,50.19685,2,68.700787,19.586614,1748,0.0,0.00,i-80,1112275155501,NaN,Car
11035519,1734,1113438174800,17.396,146.017,6042830.517,2133214.784,50.19685,2,68.438320,-20.045932,1748,0.0,0.00,i-80,1112275155701,NaN,Car
11091734,1748,1113438171700,17.625,74.564,6042839.571,2133143.858,58.39895,2,80.774278,0.000000,1734,0.0,0.00,i-80,1112275152601,NaN,Car
11156449,1734,1113438174900,17.397,148.056,6042830.267,2133216.797,50.19685,2,67.224409,-9.547244,1748,0.0,0.00,i-80,1112275155801,NaN,Car
11595872,1734,1113438171900,17.373,79.564,6042838.706,2133148.789,50.19685,2,82.020997,0.000000,1748,0.0,0.00,i-80,1112275152801,NaN,Car


In [142]:
filtered_vClass_U= filtered_ngsim_U[['Vehicle_ID','v_Class']].drop_duplicates().sort_values('v_Class')
v_Class_M_U     =set(filtered_vClass_U[(filtered_vClass_U['v_Class'] == 1 )  ]['Vehicle_ID'])
v_Class_C_U     =set(filtered_vClass_U[(filtered_vClass_U['v_Class'] == 2 )  ]['Vehicle_ID'])
v_Class_HV_U    =set(filtered_vClass_U[(filtered_vClass_U['v_Class'] == 3 )  ]['Vehicle_ID'])


filtered_vClass_I= filtered_ngsim_I[['Vehicle_ID','v_Class']].drop_duplicates().sort_values('v_Class')
v_Class_M_I     =set(filtered_vClass_I[(filtered_vClass_I['v_Class'] == 1 )  ]['Vehicle_ID'])
v_Class_C_I     =set(filtered_vClass_I[(filtered_vClass_I['v_Class'] == 2 )  ]['Vehicle_ID'])
v_Class_HV_I    =set(filtered_vClass_I[(filtered_vClass_I['v_Class'] == 3 )  ]['Vehicle_ID'])

In [143]:
result = []

for i in filtered_ngsim_U.index:
    if filtered_ngsim_U['Preceding'][i] in v_Class_C_U:
        result.append('Car')
    elif filtered_ngsim_U['Preceding'][i] in v_Class_HV_U:
        result.append('Heavy Vehicle')
    elif filtered_ngsim_U['Preceding'][i] in v_Class_M_U:
        result.append('Motorcycle')
    else: 
        result.append(np.NaN)
filtered_ngsim_U['Preceding_Vehicle_Class'] = result

result = []

for i in filtered_ngsim_I.index:
    if filtered_ngsim_I['Preceding'][i] in v_Class_C_I:
        result.append('Car')
    elif filtered_ngsim_I['Preceding'][i] in v_Class_HV_I:
        result.append('Heavy Vehicle')
    elif filtered_ngsim_I['Preceding'][i] in v_Class_M_I:
        result.append('Motorcycle')
    else: 
        result.append(np.NaN)
filtered_ngsim_I['Preceding_Vehicle_Class'] = result

In [144]:
''' this code is time consuming.
for i in filtered_ngsim_U.index:
    if filtered_ngsim_U['Preceding'][i] in v_Class_C_U:
        filtered_ngsim_U['Preceding_Vehicle_Class'][i] = 'Car'
    elif filtered_ngsim_U['Preceding'][i] in v_Class_HV_U:
        filtered_ngsim_U['Preceding_Vehicle_Class'][i] = 'Heavy Vehicle'
    elif filtered_ngsim_U['Preceding'][i] in v_Class_M_U:
        filtered_ngsim_U['Preceding_Vehicle_Class'][i] = 'Motorcycle'


for i in filtered_ngsim_I.index:
    if filtered_ngsim_I['Preceding'][i] in v_Class_C_I:
        filtered_ngsim_I['Preceding_Vehicle_Class'][i] = 'Car'
    elif filtered_ngsim_I['Preceding'][i] in v_Class_HV_I:
        filtered_ngsim_I['Preceding_Vehicle_Class'][i] = 'Heavy Vehicle'
    elif filtered_ngsim_I['Preceding'][i] in v_Class_M_I:
        filtered_ngsim_I['Preceding_Vehicle_Class'][i] = 'Motorcycle'
'''

" this code is time consuming.\nfor i in filtered_ngsim_U.index:\n    if filtered_ngsim_U['Preceding'][i] in v_Class_C_U:\n        filtered_ngsim_U['Preceding_Vehicle_Class'][i] = 'Car'\n    elif filtered_ngsim_U['Preceding'][i] in v_Class_HV_U:\n        filtered_ngsim_U['Preceding_Vehicle_Class'][i] = 'Heavy Vehicle'\n    elif filtered_ngsim_U['Preceding'][i] in v_Class_M_U:\n        filtered_ngsim_U['Preceding_Vehicle_Class'][i] = 'Motorcycle'\n\n\nfor i in filtered_ngsim_I.index:\n    if filtered_ngsim_I['Preceding'][i] in v_Class_C_I:\n        filtered_ngsim_I['Preceding_Vehicle_Class'][i] = 'Car'\n    elif filtered_ngsim_I['Preceding'][i] in v_Class_HV_I:\n        filtered_ngsim_I['Preceding_Vehicle_Class'][i] = 'Heavy Vehicle'\n    elif filtered_ngsim_I['Preceding'][i] in v_Class_M_I:\n        filtered_ngsim_I['Preceding_Vehicle_Class'][i] = 'Motorcycle'\n"

In [145]:
filtered_ngsim_U['Vehicle_combination']= filtered_ngsim_U['Preceding_Vehicle_Class'] + '-' + filtered_ngsim_U['v_Class_Name']
filtered_ngsim_I['Vehicle_combination']= filtered_ngsim_I['Preceding_Vehicle_Class'] + '-' + filtered_ngsim_I['v_Class_Name']

Vehicle_ID                 4098933
Global_Time                4098933
Local_X                    4098933
Local_Y                    4098933
Global_X                   4098933
Global_Y                   4098933
v_length                   4098933
v_Class                    4098933
v_Vel                      4098933
v_Acc                      4098933
Preceding                  4098933
Space_Headway              4098933
Time_Headway               4098933
Location                   4098933
Relative_Time              4098933
Preceding_Vehicle_Class    3915011
v_Class_Name               4098933
Vehicle_combination        3915011
dtype: int64

In [179]:
filtered_ngsim= pd.concat([filtered_ngsim_U , filtered_ngsim_I])


In [183]:
print(filtered_ngsim_U.shape[0])
print(filtered_ngsim_I.shape[0])
print(filtered_ngsim.shape[0])

4098933
4566387
8665320


In [184]:
stringpath= str(p)[0:str(p).rfind('\\')] + '\\data'
ngsimfilteredfile=stringpath + '\\' + 'Cleaned_NGSIM_Data_Part_1.csv'
filtered_ngsim.to_csv(ngsimfilteredfile, index = False)

In [200]:
#Car 1734 and 1748 are racing among each other and overtaking each other continuously 

filtered_ngsim[((filtered_ngsim['Space_Headway'] == 0 ) & ((filtered_ngsim['Preceding'] > 0 )  ) )].sort_values('Relative_Time')

,Vehicle_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,v_Acc,Preceding,Space_Headway,Time_Headway,Location,Relative_Time,Preceding_Vehicle_Class,v_Class_Name,Vehicle_combination
9803270,1394,1113437169900,76.566,444.910,6042852.523,2133518.460,48.556430,2,20.078740,0.000000,1407,0.0,0.00,i-80,1112274150801,Car,Car,Car-Car
5910808,2506,1113437493200,53.343,380.297,6042837.478,2133451.781,46.916010,2,37.828084,-0.853018,2512,0.0,0.00,i-80,1112274474101,Car,Car,Car-Car
6759942,2491,1113437558500,17.161,598.884,6042774.601,2133664.163,50.196850,2,37.171916,0.393701,2486,0.0,0.00,i-80,1112274539401,Car,Car,Car-Car
9365776,572,1113437894100,17.928,1485.487,6042644.407,2134541.664,46.916010,2,0.000000,0.000000,566,0.0,9999.99,i-80,1112274875001,Car,Car,Car-Car
11091734,1748,1113438171700,17.625,74.564,6042839.571,2133143.858,58.398950,2,80.774278,0.000000,1734,0.0,0.00,i-80,1112275152601,Car,Car,Car-Car
9153569,1734,1113438171800,17.373,77.065,6042839.014,2133146.309,50.196850,2,82.020997,0.000000,1748,0.0,0.00,i-80,1112275152701,Car,Car,Car-Car
11595872,1734,1113438171900,17.373,79.564,6042838.706,2133148.789,50.196850,2,82.020997,0.000000,1748,0.0,0.00,i-80,1112275152801,Car,Car,Car-Car
11627140,1734,1113438173600,17.377,120.537,6042833.658,2133189.478,50.196850,2,77.099738,-23.687664,1748,0.0,0.00,i-80,1112275154501,Car,Car,Car-Car
10074152,1734,1113438174600,17.393,141.704,6042831.057,2133210.438,50.196850,2,68.700787,19.586614,1748,0.0,0.00,i-80,1112275155501,Car,Car,Car-Car
11035519,1734,1113438174800,17.396,146.017,6042830.517,2133214.784,50.196850,2,68.438320,-20.045932,1748,0.0,0.00,i-80,1112275155701,Car,Car,Car-Car


In [199]:
filtered_ngsim[((filtered_ngsim['Space_Headway'] == 0 ) & (filtered_ngsim['Preceding'] > 0 )  & (filtered_ngsim['Vehicle_ID'] != 1748 )  & (filtered_ngsim['Vehicle_ID'] != 1734 )  )].sort_values('Relative_Time')

,Vehicle_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,v_Acc,Preceding,Space_Headway,Time_Headway,Location,Relative_Time,Preceding_Vehicle_Class,v_Class_Name,Vehicle_combination
9803270,1394,1113437169900,76.566,444.910,6042852.523,2133518.460,48.556430,2,20.078740,0.000000,1407,0.0,0.00,i-80,1112274150801,Car,Car,Car-Car
5910808,2506,1113437493200,53.343,380.297,6042837.478,2133451.781,46.916010,2,37.828084,-0.853018,2512,0.0,0.00,i-80,1112274474101,Car,Car,Car-Car
6759942,2491,1113437558500,17.161,598.884,6042774.601,2133664.163,50.196850,2,37.171916,0.393701,2486,0.0,0.00,i-80,1112274539401,Car,Car,Car-Car
9365776,572,1113437894100,17.928,1485.487,6042644.407,2134541.664,46.916010,2,0.000000,0.000000,566,0.0,9999.99,i-80,1112274875001,Car,Car,Car-Car
3595333,2871,1118847820000,7.631,547.070,6451508.896,1872991.251,42.650919,2,31.692913,0.000000,2876,0.0,0.00,us-101,1117684800901,Car,Car,Car-Car
5900431,2964,1118847852000,4.868,309.729,6451336.355,1873154.258,54.133858,2,28.182415,-36.745407,2956,0.0,0.00,us-101,1117684832901,Car,Car,Car-Car
76361,99,1118847948500,12.942,1420.624,6452132.084,1872437.889,27.887139,1,187.007874,-0.688976,92,0.0,0.00,us-101,1117684929401,Car,Motorcycle,Car-Motorcycle
107058,806,1118848229800,6.699,796.216,6451666.005,1872853.511,47.572178,2,30.085302,0.000000,814,0.0,0.00,us-101,1117685210701,Car,Car,Car-Car
452347,366,1118848971900,12.613,835.132,6451695.487,1872819.732,22.965879,1,93.602362,19.553806,370,0.0,0.00,us-101,1117685952801,Car,Motorcycle,Car-Motorcycle


In [190]:
def sniff(df):
    with pd.option_context("display.max_colwidth", 20):
        info = pd.DataFrame()
        info['data type'] = df.dtypes
        info['percent missing'] = df.isnull().sum()*100/len(df)
        info['No. unique'] = df.apply(lambda x: len(x.unique()))
        #info['Min Value'] = df.apply(lambda x: np.nanmin(x))
        #info['Max Value'] = df.apply(lambda x: np.nanmax(x))
        info['unique values'] = df.apply(lambda x: x.unique())
        return info.sort_values('data type')

sniff(filtered_ngsim)

,data type,percent missing,No. unique,unique values
Vehicle_ID,int64,0.000000,3233,"[515, 2127, 1033, 1890, 744, 496, 946, 1779, 2..."
Global_Time,int64,0.000000,57405,"[1118848075000, 1118847624800, 1118848324700, ..."
Relative_Time,int64,0.000000,57405,"[1117685055901, 1117684605701, 1117685305601, ..."
Preceding,int64,0.000000,3228,"[500, 2124, 1029, 1882, 740, 510, 936, 1769, 2..."
v_Class,int64,0.000000,3,"[2, 3, 1]"
Time_Headway,float64,0.000000,19727,"[5.11, 1.3, 0.92, 2.27, 1.54, 2.09, 2.69, 3.8,..."
Space_Headway,float64,0.000000,40809,"[390.74803149606294, 160.498687664042, 127.329..."
v_Acc,float64,0.000000,2239,"[6.725721784776902, 36.745406824146976, 0.3280..."
v_Vel,float64,0.000000,8986,"[76.4763779527559, 123.0971128608924, 137.7624..."
v_length,float64,0.000000,276,"[42.650918635170605, 44.29133858267716, 78.740..."
